# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='../chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 武漢肺炎疫情延燒　翔宇：營運無重大影響

▲翔宇旗下品牌。（圖／翔宇提供）
記者姚惠茹／台北報導
翔宇生醫（6539）今（31）日宣布，針對武漢肺炎疫情延燒，由於大陸主要營運地區為上海，營收比重不高，因此武漢肺炎對公司整體營運並無重大影響，目前上海子公司皆將依循法定假期規定及地方防疫狀況，調整門店營業時間及開業日期，預計將在2月10日復工。
翔宇表示，農曆過年期間，翔宇已緊急採取回報制度掌握，全體員工狀況目前一切正常，因此武漢肺炎對公司整體營運並無重大影響，而上海子公司皆將依循法定假期規定及地方防疫狀況，調整門店營業時間及開業日期，在保障顧客和員工的健康與安全的前提下，恢復營運。
翔宇指出，旗下DV保健食品包含喜樂纖、醇養妍等，由於其產品配方中擁有綠茶兒茶素、白藜蘆醇、野櫻莓等營養成份，凸顯翔宇DV保健食品的產品獨特性，並在各大銷售通路包括電商平台等均有優異回購率的表現，有機會挹注翔宇今年營運遞出正向成長的成績單。
展望未來，翔宇仍持續深化產品質量與廣度，增加商品的整合效益，滿足不同消費者客群需求，更強化DV於兩岸美容保養市場的品牌力，整合內外資源與槓桿縱橫整合效益，優化SPA通路單店營運表現為主要營運方針，奠定翔宇公司穩定營收成長的基石，創造公司營運穩健向上的態勢。
----------------------------------------------------------------------

[1] 台灣東洋晚期肺癌口服藥　取得非小細胞肺癌健保給付

▲台灣東洋董事長林全。（圖／資料畫面）
記者姚惠茹／台北報導
----------------------------------------------------------------------

[2] 苗縣府標售「竹科竹南基地周邊地區特定區」　區段徵收可建地

▲苗栗縣政府將辦理「竹科竹南基地周邊地區特定區」標售，歡迎民眾參與投標。（圖／苗栗縣政府提供）
記者黃孟珍／苗栗報導
苗栗縣政府定於2020年2月6日辦理「竹科竹南基地周邊地區特定區」區段徵收開發案科技商務區及住宅區土地標售開標作業，本特定區內建案林立，擁有完整街廓、完善公共設施及交

▲車燈。（圖／pixabay）
記者楊絡懸／新北報導
LED車燈模組老字號「麗清科技」（3346）針對中國營運地區表示，面對中國新型冠狀病毒「武漢肺炎」感染情形，旗下武漢廠為新廠區，目前仍處於初期小批量產階段，占公司整體營收比重不到1%，對整體營運影響不大。
麗清科技表示，在農曆過年期間，已緊急採取回報制度掌握公司全體員工，目前一切正常，也將配合所在地政府及車廠的延遲復工通知，麗清武漢廠預計於2月14日開工，上海及東莞廠則復工時間為2月10日，目前仍持續密切注意疫情狀況，並與工廠密切聯繫，待復工後協調產能，儘速出貨。
近期申報回台設廠的麗清進一步表示，公司在中國，布局包含上海、東莞、武漢等3個廠區，積極將自動化生產導入於單一產品的生產流程中，目前已具有高自動化生產技術，因此，可隨時依據主要客戶的拉貨需求，快速調配各廠區之生產產能並出貨。
▲LED車燈模組老字號「麗清科技」。（圖／翻攝自「麗清汽車」形象影片）
展望未來，麗清雖對目前中國整體新車銷售市場持審慎保守的看法，但受惠於新車導入LED車燈的滲透率提升，有助於公司保持良好的訂單能見度，加上近期台灣政府推動「台商回台」投資政策，麗清為因應主要客戶的生產需求與未來營運發展，已在桃園觀音規劃興建智慧化工廠，整廠輸入高速自動化設備。
麗清表示，除了以期已接獲之新訂單挹注產能利用率提升外，也可望在彈性調整兩地生產基地的產能配置下，爭取更多外銷美國地區訂單的合作機會，並對整體營運帶來正面的挹注。
【延伸閱讀】
----------------------------------------------------------------------

[15] 無需投資數千萬買地蓋農舍　宜蘭小坪數農地也能享受農耕樂

▲▼數百萬元，宜蘭小坪數農地讓你享受渡假與農耕樂，東森房屋宜蘭大學加盟店長賴富厚是強烈推荐。（圖／記者游芳男攝，下同）
記者游芳男／宜蘭報導
國5通車後，好山好水的宜蘭，成為大台北地區後花園，而長年在都市叢林，為事業家庭打拚奮鬥的退休高齡朋友，想小投資來享受田園農耕及渡假之樂，東森房屋宜蘭大學加盟店長賴富厚強烈推荐宜蘭非老農持有的價格較低，且是300至500坪的小坪數農地，是不錯選擇，300至500坪非老農持有的小農地，雖不能興建農舍，但只要有種蔬業、果樹、水稻等實質農耕情形，就可申設資材室、農機室、加工室等建

▲市場擔憂武漢疫情恐削弱石油消費量成長，專家預估下周汽柴油零售價格可望調降。（圖／記者林敬旻攝）
記者林淑慧／台北報導
武漢肺炎疫情持續延燒，衝擊交通旅運需求，近期國際油價回跌，專家預估下週汽、柴油零售價格每公升約各調降0.9、1.0元，預估98無鉛汽油每公升將跌破30元大關，想加油的民眾不妨再等等，替自己省點荷包。
依據專家估算，調整後的參考零售價格分別為92無鉛汽油每公升25.9元、95無鉛汽油每公升27.4元、98無鉛汽油每公升29.4元、超級柴油每公升23.7元。
油價專家表示，武漢疫情持續延燒，市場擔憂武漢疫情恐削弱石油消費量成長，近期油價回跌；累算至1月30日之7D3B週均價為58.58美元，較上週63.32下跌4.74美元。新臺幣兌美元匯率30.277較上週29.988貶值0.289元。
依浮動油價調整機制作業原則，以及不考量亞鄰競爭國最低價及平穩措施的條件下，預估下週汽、柴油零售價格每公升約各調降0.9、1.0元，實際調整金額請於週日中午12時詳見台灣中油全球資訊網暨經濟部網站。
----------------------------------------------------------------------

[27] 全球投資人瘋買黃金！分析師爆關鍵四大主因

記者陳心怡／外電報導
全球各地投資人爭先恐後擁抱黃金，根據《金融時報》報導，由於去年黃金擔保的指數股票型基金（ETF）獲得192億美元資金流入後，目前黃金數量持有量已攀升至七年高點，分析師認為是因投資人擔心主要市場成長減緩、地緣政治風險升溫、傳統避險資產不再安全，以及另一大重要原因，負殖利率債券規模擴大。
目前全球負殖利率債券規模已超過13兆美元，如果債券買家被要求支付持有債券到期的權利利息費用，那麼黃金吸引力反而會大增。
Sprott黃金股票基金共同經理人海瑟威（John Hathaway）說：「可已看到資金正流向黃金，它是一種全球趨勢。一般的避險資產已無法確保安全，因此更多人搶著要黃金。」
然而在本季度，金價已升至2013年以來的最高水平，每盎司超過1,600美元。根據世界黃金協會（World Gold Council）的數據，自2015年以來，全球黃金ETF的黃金持有量自2015年來已增加逾一倍至1,410億美元，這也幫助SPDR黃金信託基金躍升成為美國第十大ETF。
在黃

▲受武漢肺炎疫情影響，台股重挫696.97點。（圖／記者湯興漢攝）
記者周康玉／台北報導
金鼠年開紅盤就給投資人一個大震撼，受到中國武漢肺炎疫情影響，台股鼠年第一個一開盤交易日就開低走低，收盤收在11421.74點，大跌696.97點，跌幅5.75%，創下單日最大跌點。市場法人表示，肺炎影響短期市場信心及總體經濟需求，參考過往經驗跌勢可能為期一個月，非經濟面因素造成下跌往往伴隨V型反轉，後續密切觀察買進加碼時點。
法人建議，不妨以台股基金來介入布局，一來透過專業經理人的篩選來降低選股的不確定性，二來也有助分散投資風險，掌握台股不同產業的投資契機。市場普遍預期疫情仍在高峰期，武漢1月23日封城，按照7~14天潛伏期，2/8元宵節前後是確診數高峰期，短期上市場風險偏好趨於謹慎。
群益投信表示，相較2003年SARS期間，台股跌幅10%左右，SARS疫情期間台股高低點落差約20%，惟低點4044點後漲至7135點；台股30日已單日大跌5.75%，市場恐慌情緒已釋放不少，加上本次全球防疫措施相對嚴拒謹慎，相關影響可望降低。
群益投信表示，台股加權指數短期下跌難免，2000年來有10次單日下跌超過5.5%，不過之後期間平均表現都為正報酬；中長期來看，FED在內的全球央行仍力行寬鬆政策，整體市場資金充沛、景氣仍為溫和復甦；台灣受惠政策獎勵海外資金與台商回流、轉單效應，不論實質景氣面、企業資本投入、與金融市場投資氛圍較為正面，2020年台股仍可望挑戰新高。
群益馬拉松基金經理人沈萬鈞指出，日後需觀盤重點包括中國肺炎疫情需嚴密觀察是否引起全球病例數爆發，影響總體經濟展望。2月中旬及下旬美國及中國經濟數據，預期可能受肺炎小幅影響而下修。此外，2月下旬台股上市櫃公司陸續發表年報以及舉辦法說會，關注各產業展望。
沈萬鈞表示，2月底巴塞隆納全球MWC展，科技產品及新應用仍具話題性。產業部分，半導體先進製程、5G建置及新技術應用展望仍佳且資金寬鬆持續，中長期仍看多，2020年上游半導體包括CPU/GPU/server皆有新平台推出，晶圓代工在高階製程與封測廠在高階封測皆持續大幅擴充資本支出，相關設備廠商可望迎來數年大多頭行情。整體而言，仍看好半導體、5G、IC設計等族群。
------------------------------------------------------------

▲彰銀爆出理專挪用客戶資金事件，金管會表示，該名理專涉及挪用4名客戶資金長達10年半時間，合計挪用資金約為6,300萬元。（圖／記者吳靜君攝）
記者紀佳妘／台北報導
理專A錢事件又一樁！金管會今（30）日表示，彰化銀行北台中分行一名理專從2009年底到2019年6月，涉及挪用客戶資金，一共影響4名客戶、挪用資金約達6,300萬元。
金管會銀行局副局長莊琇媛表示，彰化銀行有一名非受害客戶投訴該名理專，有保管印章、保證獲利及收到不認識的人匯款等情況，隨即彰銀稽核單位介入查後發現，這名理專與客戶有資金異常的情況。
莊琇媛指出，這名理專涉及挪用資金約達6,300萬元，其中未依客戶指示交易，且有挪為他用為4,000萬元，另與客戶借貸1,600萬元、異常資金往來590萬元。
莊琇媛說，彰銀發現該名理專有資金異常狀況，先前已向金管會通報重大偶發，目前彰銀已完成「陳述意見」，金管會將依照陳述意見進行處理。
此外，金管會農曆春節前才重罰2019年爆發的永豐銀行、玉山銀行理專分別挪用客戶金額高達7,000萬元、3,000萬元，銀行局各祭出1,200萬元罰單，也是銀行法加重罰鍰上路後的首例，外界關注彰化銀這起理專盜用資金案，恐也會依照相關案例開罰。
----------------------------------------------------------------------

[54] 台指期重挫741點創開紅盤日最慘紀錄！　分析師：賣壓未宣洩完、明日續跌

▲台指期2020年開紅盤日重挫。（圖／取自期交所）
記者余弦妙／台北報導
春節連假過後，台指期今（30）日開紅盤恢復交易，但受武漢肺炎影響，一開盤就大跌341點，來到11,770 點，跌幅高達2.8%，但這還不是谷底，盤中一路下跌，終場以11,370點作收，一口氣大跌741點，創下史上單日最大跌點紀錄。
以一口台指期多單來說，投資人一天就虧掉14.8萬元，保證金都不夠墊，還要被追繳。但相對來說，台指期空單投資人，今天可就是大獲全勝，為自己賺得一個年後大紅包。
王榮旭表示，今天外資在期貨與現貨都大賣，且目前盤後電子盤仍持續大跌，因此明天會持續走跌的機率相當高，因為通常只要外資期、現貨同步賣的話就會呈現短線趨勢成形，今天調節的口數也相對較多，外資在台指期賣超了1萬1,751口，淨多單剩下1萬8,239口，算是近期較低的水位

▲荒廢12年的彰化市喬友大樓2月重新營運。（圖／翻攝自林世賢臉書粉專）
記者唐詠絮／彰化報導
位於彰化火車站前，彰化市的地標15F「喬友大廈」，在經歷大火及管理不善，整整荒廢了12年，如今好消息傳來，大廈管委會已經招商並在內部裝潢施工，準備過完農曆春節後2月正式開幕營運，重振商機，房仲業者分析，這隻金雞母的重生，將可以帶入另一波的人流與金流，更可帶動彰化火車站前的不動產買氣。
▲從頂樓遠眺可以看到彰化市扇型車站。（圖／翻攝自林世賢臉書粉專）
彰化市長林世賢當年還是議員時，與縣府建設處使管科長去會勘時，還一度感概，矗立在彰化市火車站前的喬友大廈，在1993年開幕時，這棟地下3樓、地上15樓的建物，包括有百貨商場、KTV、遊樂世界、冰宮、旅館等，繁榮至極，一度是除八卦山大佛之外，彰化市最熱鬧的地標。
站上喬友頂樓可以近可眺望彰化火車站及扇形車庫，八卦山大佛，天氣好時遠可眺望大肚山及彰濱離岸風機，環伺四周地理條件相當的優越，林世賢指出，由於管理問題與遭受祝融，業者陸續撤出，喬友大廈不再耀眼，最後僅剩一樓在營業，被譏為廢墟，堪稱是彰化市民心中的痛，還要求縣府應佔輔導立場、協助所有權人、復甦活化彰化火車站商業。
▲遠觀還能看到彰濱。（圖／翻攝自林世賢臉書粉專）
大廈所有權人組管委會展開重修活化自救，B1-B3規劃地下停車場外，還有東南亞異國等料理美食，而其他樓層則出租世界健身中心、補習班及AI智能無人櫃台旅館等，重新營運後喬友大廈，一定可以帶給市民一個充滿健康朝氣有活力的新氣象。
----------------------------------------------------------------------

[66] 春節ATM交易金額較去年減少逾百億　放假首日交易量最多

▲ 央行統計春節期間ATM交易情形。（示意圖／記者賴文萱攝）
記者李瑞瑾／綜合報導
央行針對今年春節假期ATM跨行交易資料分析，假期首日之跨行交易金額為最高，跨行提款金額與跨行轉帳金額皆較去（108）年減少超過百億，央行表示，主要係因今年春節假期較108年少2日的緣故。
央行針對今年春節假期ATM跨行交易資料分析，假期首日之跨行交易金額為最高，假期中則下降，至假期最後1日才又增加，與歷年相同。
央行為確保農曆春節期間財金公司ATM跨行系統運作順暢，便利民眾提款及轉帳，已於春節前促請各金融機構

▲彰化市開化寺前路段蟬聯4年地王寶座。（圖／記者唐詠絮翻攝）
記者唐詠絮／彰化報導
彰化縣地價及標準地價評議委員會公告土地現值及公告地價作業，全縣最高地價區段仍由開化寺旁民族路兩側–中華路及太平街間之地段蟬聯，公告土地現值為每平方公尺16.1萬元，換算每坪53.2萬元，2017年凍漲至今，蟬聯4年穩坐地王寶座。
至於全縣地價最低的土地區段，則位在西南角的大城鄉中城段，屬於海埔新生地，公告現值每平方公尺僅100元，每坪土地價330元。
地政處表示，2020年地價調查作業期間（2018年9月2日至2019年9月1日），全縣不動產實價登錄買賣案件交易量與前期相當，成交價格及地價調查結果，亦有微幅上漲。整體而言不動產市場呈現「量價平穩」趨勢。
目前交易較熱絡區域包括彰化縣彰化市南興國小、精誠中學附近新建案及後火車站區域、金馬路、都市計畫外東區彰南路附近區域、員林市184市地重劃區範圍、鹿港鎮都市計畫內邊緣區、和美鎮自辦市地重劃區等及計畫道路陸續完工通車帶動沿線鄉鎮市之區域。
地政處也提醒民眾，公告土地現值係為課徵增值稅、遺產稅及贈與稅的依據，民眾若無買賣或贈與等不動產移轉情況，將不會因現值調整而影響，受理申報地價期間至1月31日（星期例假日不受理申報），並提供現場、網路申報地價方式，土地所有權人或管理人可擇一方式至轄區地政事務所辦理。另依據平均地權條例第16條規定，土地所有權人得參考2020年公告地價在增減20％以內申報地價；如未辦理申報地價者，則按公告地價80％作為申報地價，該申報地價並依法作為課徵地價稅之基準。
----------------------------------------------------------------------

[78] 開工試手氣！大樂透1.6億頭獎＋159組100萬紅包等你拿　3種買法易中大獎

▲今天是開工日，大樂透頭獎上看1.6億元，同時加碼開出159組春節百萬大紅包。（示意圖／記者呂佳賢攝）
記者紀佳妘／台北報導
台灣彩券公司今（2020）年農曆春節電腦型彩券加碼總獎金8億元，其中春節7天連續假期，大樂透一共開出2次頭獎、送出201組100萬元，今（30）日新春開工日，大樂透頭獎上看1.6億元，也將加碼續開100萬元獎項。此外，威力彩自1月20日加碼2億元後，頭獎尚未開出，今晚頭獎上看3.8億元，也是今年最高的頭獎。


▲蘋果昨（29）日發表2020會計年度第一季財報，營收、EPS雙雙創新高。圖為蘋果執行長庫克。（圖／達志影像／美聯社）
記者王曉敏／綜合報導
蘋果昨（29）日發表2020會計年度第一季財報，營收、EPS雙雙創新高，連同下季財測均超出華爾街預期。據蘋果執行長庫克（Tim Cook）的說法，該公司上季亮眼的財報表現主要得歸功於3大功臣：iPhone 11、AirPods Pro及Apple Watch。
庫克在新聞稿中表示，「由於蘋果的iPhone 11及iPhone 11 Pro系列需求強勁及服務、可穿戴設備銷售創下歷史新高，我們很高興地宣布，蘋果創下單季營收的新紀錄。」「在假期季度中，我們活躍的設備安裝量在每個地理區域均有所增長，目前已超過15億。我們認為這是對客戶滿意度、敬業度和忠誠度的有力證明，也是我們全面發展的重要推動力。」
而針對武漢肺炎疫情，庫克在法說會上表示將對蘋果前景帶來「不確定性」。蘋果Q2財測預計營收將介於630億美元至670億美元間，鑒於疫情影響，蘋果在武漢地區的一些供應商延後開工，目前蘋果已關閉一家位於中國大陸的門市，並縮減零售商店的營業時間，同時也限制員工出差。庫克表示：「情況正在浮現，我們將持續收集資料並且密切關注。」
----------------------------------------------------------------------

[89] 宏碁取消團拜！施振榮雲端拜年　防止疫情擴散大秀「線上會議系統」

▲宏碁運用aiSage攜手視訊軟體公司Zoom合作開發的線上會議系統。（圖／宏碁提供）
記者姚惠茹／台北報導
宏碁（2353）創辦人施振榮今（30）日舉雲端團拜，因應武漢肺炎疫情爆發，大秀宏碁運用aiSage攜手視訊軟體公司Zoom合作開發的線上會議系統，主要是將宏碁硬體與雲端整合推出的企業視訊會議解決方案，目前正在試驗中，預計3個月後推出。
施振榮每年新春都在自家與媒體舉辦團拜，今年受到武漢肺炎疫情爆發的因素影響，施振榮特別開啟「視訊拜年」的新模式，並與夫人葉紫華及長子施宣輝，共同使用以宏碁旗下的AI邊緣運算裝置aiSage為基礎架構的多方視訊會議，舉辦一場別開生面的雲端團拜。
施振榮表示，視訊是對付武漢肺炎疫情擴散的新方法，這套系統是宏碁與視訊軟體公司Zoom合作開發的企業視訊會議解決方案，最高可以支援4